In [1]:
import torch
from collections import deque
import random
import copy
from torch.autograd import Variable
import numpy as np



In [2]:
reward_pc = torch.tensor([1, 5 , 5,3,-1, 5, 5,3,1,-2 ])
reward_npc = torch.tensor([0,-2,-3,-1,0, -2, -3, -1,1])

#index = id-1


In [3]:
class Environment():
    def __init__(self):
        self.state=(0,0)
        self.superState = 0
    def PC_action(self,action_transformed):
        if self.superState ==0:
            return 1 
            ## code to ask input from the user
            # transform into the id

        elif self.superState == 1:
            if action_transformed == 3:
                return 2
            elif action_transformed ==5:
                return -1
            else:
                lister = [3,4]
                return random.choice(lister)

        elif self.superState ==2:
            return 6

        elif self.superState ==3 :
            lister = [7,8,9,10]
            return random.choice(lister)   

        else :
            return -1
        
    def return_reward(self):
        return reward_npc[self.state[0]-1] + reward_pc[self.state[1]-1]


    def next_state(self,action_transformed ):
        pc_action = self.PC_action(action_transformed)
        if pc_action ==-1:
            self.state =(-1,-1)
            return self.state
        self.state = (action_transformed,pc_action)
        if self.superState == 0:
            
            self.superState =1
        elif self.superState == 1:

            if action_transformed == 2:

                self.superState = 2
            elif pc_action == 6:
                self.superState =4

            else :
                self.superState=3


        elif self.superState ==2:
            self.superState = 3

        elif self.superState ==3:
            self.superState = 4

        else:
            self.superState = 4
        
        return self.state
        
                           

        

            



        




        
        
# This is just a dummy environment 

In [4]:
#Instantiata a environment

env = Environment()


In [5]:
class DQN():
    def __init__(self, n_state, n_action, n_hidden=50, lr=0.05):
        self.criterion = torch.nn.MSELoss()

        self.model = torch.nn.Sequential(
            torch.nn.Linear(n_state,n_hidden),
            torch.nn.ReLU(),
            torch.nn.Linear(n_hidden, n_action)
        )

        self.optimizer = torch.optim.Adam(self.model.parameters(),lr)

        self.model_target = copy.deepcopy(self.model)

    def target_predict(self, s):
        with torch.no_grad():
            return self.model_target(torch.Tensor(s))

    def copy_target(self):
        self.model_target.load_state_dict(self.model.state_dict())
    
    def update(self, s,y):
        y_pred = self.model(torch.Tensor(s))
        loss = self.criterion(y_pred, Variable(torch.Tensor(y)))
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

    def predict(self, s):
        with torch.no_grad():
            return self.model(torch.Tensor(s))


                
    def replay(self, memory, replay_size, gamma):
        if len(memory) >= replay_size:
            replay_data = random.sample(memory , replay_size)
            states =[]
            td_targets =[]
            for state, action, next_state, reward, is_done in replay_data:
                states.append(state)
                q_values = self.predict(state).tolist()
                if is_done:
                    q_values[action] = reward
                else:
                    q_values_next = self.target_predict(next_state).detach()
                    q_values[action] = reward + gamma* torch.max(q_values_next).item()

                td_targets.append(q_values)

            states = np.array(states)
            self.update(states,td_targets)

       



In [6]:
def gen_epsilon_greed_policy(estimator,epsilon, n_action ):
    def policy_function(state):
        if random.random() < epsilon:
            return random.randint(0,n_action-1)

        else:
            q_values = estimator.predict(state)
            return torch.argmax(q_values).item()

    return policy_function


In [7]:
n_state = 2
n_action = 3
n_hidden =50
lr = 0.01
dqn = DQN(n_state, n_action, n_hidden, lr)

In [8]:
memory = deque(maxlen=10000)

In [9]:
def q_learning(env, estimator , n_episode, replay_size, target_update=10, gamma=1.0, epsilon=0.1 , epsilon_decay=0.99):
    for episode in range(n_episode):
        if episode % target_update ==0 :
            estimator.copy_target()
        policy = gen_epsilon_greed_policy(estimator, epsilon, n_action)
        env.state=(0,0)
        env.superState = 0
        state = env.state

        is_done = False
        while not is_done:
            action = policy(state)
            next_state = env.next_state(action)
            reward =env.return_reward()
            total_reward_episode[episode] += reward

            if env.state == (-1, -1):
                is_done = True

            memory.append((state, action, next_state, reward, is_done))
            print(next_state)
            if is_done:
                print("end episode")
                break

            estimator.replay(memory, replay_size, gamma)
            state = next_state

        print('episode: {}, total reward : {}, epsilon:{}'.format(episode, total_reward_episode[episode], epsilon))
        epsilon = max(epsilon * epsilon_decay, 0.01)











In [10]:
n_episode = 1000
replay_size = 20 

target_update = 10 


In [11]:
total_reward_episode = [0] * n_episode
q_learning(env, dqn, n_episode, replay_size, target_update, gamma=0.9, epsilon = 1)

(2, 1)
(1, 4)
(2, 10)
(-1, -1)
end episode
episode: 0, total reward : -2, epsilon:1
(1, 1)
(1, 4)
(0, 10)
(-1, -1)
end episode
episode: 1, total reward : 3, epsilon:0.99
(1, 1)
(2, 3)
(2, 6)
(0, 7)
(-1, -1)
end episode
episode: 2, total reward : 13, epsilon:0.9801
(0, 1)
(0, 3)
(2, 8)
(-1, -1)
end episode
episode: 3, total reward : 9, epsilon:0.9702989999999999
(2, 1)
(2, 3)
(2, 6)
(1, 9)
(-1, -1)
end episode
episode: 4, total reward : 6, epsilon:0.96059601
(2, 1)
(1, 3)
(2, 7)
(-1, -1)
end episode
episode: 5, total reward : 7, epsilon:0.9509900498999999
(0, 1)
(1, 4)
(1, 10)
(-1, -1)
end episode
episode: 6, total reward : 3, epsilon:0.9414801494009999
(0, 1)
(0, 4)
(1, 8)
(-1, -1)
end episode
episode: 7, total reward : 9, epsilon:0.9320653479069899
(1, 1)
(0, 3)
(2, 8)
(-1, -1)
end episode
episode: 8, total reward : 8, epsilon:0.92274469442792
(0, 1)
(2, 4)
(2, 6)
(1, 8)
(-1, -1)
end episode
episode: 9, total reward : 9, epsilon:0.9135172474836407
(1, 1)
(0, 4)
(1, 10)
(-1, -1)
end ep